In [ ]:
-- Create the API integration for DORA
use role ACCOUNTADMIN;

create or replace api integration dora_api_integration 
api_provider = aws_api_gateway 
api_aws_role_arn = 'arn:aws:iam::321463406630:role/snowflakeLearnerAssumedRole' 
enabled = true 
api_allowed_prefixes = ('https://awy6hshxy4.execute-api.us-west-2.amazonaws.com/dev/edu_dora');


In [ ]:
show integrations;

In [ ]:
-- Create the UTIL_DB database (and implicitly the PUBLIC schema
create database if not exists util_db;

In [ ]:
-- Create SE greeting function
create or replace external function util_db.public.se_greeting(
      email varchar
    , firstname varchar
    , middlename varchar
    , lastname varchar)
returns variant
api_integration = dora_api_integration
context_headers = (current_timestamp,current_account, current_statement, current_account_name) 
as 'https://awy6hshxy4.execute-api.us-west-2.amazonaws.com/dev/edu_dora/greeting'
;


In [ ]:
-- Initialize the link between the SE and the Account - run this once
use role ACCOUNTADMIN;

--Please edit the SQL statement to provide your email address (@snowflake.com), first, middle (optional), and last names
--Remove the angle brackets and put single quotes around each value
--When entering your names:
--Use mixed case
--Don't use ALL CAPITAL LETTERS
--Don't use all lowercase letters
--Don't use CamelCase without spaces between your names if you have more than one
--You must enter both a first and last name, and single letters don't count as names
--Middle names are optional, so if you don't want to provide one replace <middle name> with ''
--You can use accents or letters from any language
--You can use as many words as you want in your name, and please put spaces between them

select util_db.public.se_greeting('sagar.Kulkarni@snowflake.com', 'sAGar', '', 'KulKarni');

In [ ]:
-- Create the grader
use role accountadmin;

create or replace external function util_db.public.se_grader(        
 step varchar     
 , passed boolean     
 , actual integer     
 , expected integer    
 , description varchar) 
 returns variant 
 api_integration = dora_api_integration 
 context_headers = (current_timestamp,current_account, current_statement,current_account_name) 
 as 'https://awy6hshxy4.execute-api.us-west-2.amazonaws.com/dev/edu_dora/grader'  
;


In [ ]:
grant usage on database util_db to role public;
grant usage on schema util_db.public to role public;
grant usage on function util_db.public.se_grader(varchar,boolean,integer,integer,varchar) to role public;

In [ ]:
select se_grader(step, (actual = expected), actual, expected, description) as graded_results from (SELECT
 'DORA_IS_WORKING' as step
 ,( select 123) as actual
 ,123 as expected
 ,'Dora is working!' as description
);

